In [212]:
import pandas as pd
import numpy as np
import re as r
import seaborn as sns
from datetime import datetime


In [213]:
#Editor input goals
art_goals = 10000
market_goals = 10000
creativity_goals = 10000
vc_goals = 10000

goals = [art_goals, market_goals,creativity_goals,vc_goals]
g = dict(zip(Verticals,goals))


In [214]:
#Add Looker Editorial Sessions here
df = pd.read_csv('analytics editorial_sessions 2019-04-02T1120.csv',thousands=',')


In [215]:
#Add AT data here
projections_df = pd.read_csv('Production-All Verticals (10).csv',thousands=',')


In [216]:
#List of Verticals, you can add, subtract, or edit verticals here
Verticals = ['Art','Art Market','Creativity','Visual Culture']
agg_func = {'var': np.mean,'sessions':np.sum,'Projections':np.sum}

In [217]:
#Attempt to compare this week vs last week, haven't completed
my_date = datetime.now() # if date is 01/01/2018
year, week_num, day_of_week = my_date.isocalendar()
current_week = week_num
last_week = current_week-1
next_week = week_num+1
last_week_art_sessions = 0

In [218]:
#Filtering AT to create two tables, one with articles up to current date and one that is set for publish days after date
current_month = datetime.now().month
current_date = pd.to_datetime('today')
current_month = '2019/'+str(current_month)


In [219]:
#Cleaning up Looker table
df = df.rename(index=str,columns={"Articles Vertical":"vertical"})
df = df.rename(index=str,columns={'Articles Tags':'tag'})
df = df.rename(index=str,columns={'In-Month Sessions Sessions Count':'sessions'})
df = df.rename(index=str,columns={'Articles Slug':'slug'})

In [220]:
#Cleaning up AT table
projections_df = projections_df.fillna(0)
projections_df = projections_df.rename(index=str,columns={'Article Link':'slug'})
projections_df = projections_df.rename(index=str,columns={'Traffic Projection':'Projections'})
projections_df = projections_df.rename(index=str,columns={'Sub-Vertical':'tags'})
projections_df = projections_df.rename(index=str,columns={'ACCOUNTING: MONTH':'pub_month'})
projections_df = projections_df.rename(index=str,columns={'Publish Date':'pub'})
projections_df['pub'] = pd.to_datetime(projections_df.pub,errors='coerce')
projections_df['pub'] = pd.to_datetime(projections_df['pub'].dt.date)
projections_df['Month'] = pd.to_datetime(projections_df.Month,errors='coerce')
projections_df['Month'] = pd.to_datetime(projections_df['Month'].dt.date)
projections_df = projections_df.replace('">15,000 sessions"',15000)
projections_df = projections_df.replace('"<5,000 sessions"',5000)
projections_df = projections_df.replace('"5,000-15,000 sessions"',10000)
projections_df = projections_df.drop(columns=['Angle','Admin','Field 105','Field 106'])
projections_df = projections_df[(projections_df.pub_month==current_month)]
projections_df['slug'].replace(regex=True,inplace=True,to_replace=r'https://www.artsy.net/article/',value=r'')
projections_df['slug'] = projections_df.slug.astype(object)


In [221]:
#Combination of AT and Looker Tables matched on URL
merged=pd.merge(df,projections_df,on='slug',how='inner')
merged=merged.drop(columns=['tag','Work Week','Story','Vertical','Filing Date','Month','Publish Time'])
var=""
merged['var']=var
merged['var'] = merged['sessions'] / merged['Projections']


In [222]:
#List of articles to be published, use this table for projections
new_projections_df = projections_df[(projections_df.pub>current_date)]


In [223]:
#Creation of dicts for filtering
d={}
pivot={}
summary={}
performance={}
count={}
projected_sessions={}
projected_count={}
for name in Verticals:
    d[name] = pd.DataFrame(merged[(merged.vertical==name)])
    pivot[name] = pd.DataFrame(merged[(merged.vertical==name)])
    summary[name] = pd.DataFrame(merged[(merged.vertical==name)])
    performance[name] = pd.DataFrame(merged[(merged.vertical==name)])
    count[name] = pd.DataFrame(merged[(merged.vertical==name)])
    projected_sessions[name] = pd.DataFrame(new_projections_df[(new_projections_df.Vertical==name)])
    projected_count[name] = pd.DataFrame(new_projections_df[(new_projections_df.Vertical==name)])

In [224]:
#function to create pivot tables and summary tables
def filter(dictionary):
    for name, df in d.items():
        pivot[name] = pd.pivot_table(d[name], values=['var','Projections','sessions'],index=['tags','Articles Title'], aggfunc={'var': np.mean,'sessions':np.sum,'Projections':np.sum})
        pivot[name] = pd.concat([r.append(r.agg(agg_func).rename((k, 'Total')))
           for k, r in pivot[name].groupby(level=0)]).append(pivot[name].agg(agg_func).rename(('Grand', 'Total')))
        sessions = pivot[name].loc['Grand','sessions']['Total']
        projected_sessions[name] =  projected_sessions[name]['Projections'].sum()
        total = projected_sessions[name]+sessions
        summary[name] = pd.DataFrame({0:['Current Sessions as of ' + str(current_date),'Projected Rest of Month','Total Projected','Goals','Difference','% Difference'],1:[sessions,projected_sessions[name],projected_sessions[name]+sessions,g[name],total-g[name],(total-g[name]/g[name])]})
        performance[name]=pd.DataFrame({'Count':['1.05+','.90-1.04','.50-.89','.0-.49'],'Amount':[sum((performance[name]['var']>1.04)),sum((performance[name]['var'].between(.9,1.04))),sum((performance[name]['var'].between(.5,.89))),sum((performance[name]['var'].between(0,.49)))]})
        count[name]=pd.DataFrame({'Count':['15K','5K-10K','5K'],'Amount':[sum((count[name]['Projections']==15000)),sum((count[name]['Projections']==10000)),sum((count[name]['Projections']==5000))]})
        projected_count[name]=pd.DataFrame({'Projected Count':['15K','5K-10K','5K'],'Amount':[sum((projected_count[name]['Projections']==15000)),sum((projected_count[name]['Projections']==10000)),sum((projected_count[name]['Projections']==5000))]})
       


       

In [225]:
#running function of dictionary 
filter(d)


ValueError: No objects to concatenate

In [226]:
#creates excel spreadsheet with all data summarized
with pd.ExcelWriter('article_tracker.xlsx') as writer:  # doctest: +SKIP
     for x in Verticals:
            pivot[x].to_excel(writer,sheet_name=x+'_pivot')
            performance[x].to_excel(writer,sheet_name=x+'_performance')
            count[x].to_excel(writer,sheet_name=x+'_count')
            projected_count[x].to_excel(writer,sheet_name=x+'_projected_count')
            summary[x].to_excel(writer,sheet_name=x+'_summary')
            
            
            
            
            
            
            
 
    

IndexError: index 0 is out of bounds for axis 0 with size 0